In [ ]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
import scipy.stats as st
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

np.set_printoptions(precision=2)
pd.set_option('display.max_rows', 200)

In [ ]:
#calculate confidence interval
def get_standard_diviation(values):
  mean = np.mean(values)
  n = len(values)
  s = np.sqrt(np.sum((values-mean)**2)/(n-1))

  return s # (mean-2*s, mean+2*s) 

## Load data

In [ ]:
df = pd.read_csv("C:/Users/python_test/Desktop/Speciale/df_MRI_features_12042022", sep=",", header=0)
df = df.set_index("id")
df = df.drop(["v1: id"],axis=1)

In [ ]:
#SKIP FOR FIRST VERSION (limit amount of features)
var_of_interest = ["v1: Femur (vol)","v1: JWS (approx)", "v1: Img Intensity (mean)", "v1: Tibial Intensity (mean)", "v1: Femur Intensity (mean)", 
                   "v1: Miniscus Intensity (mean)", "v1: Entropy Tibial", "v1: Entropy Meniscus","v1: Entropy Femur","v1: Closing Femur","v1: Closing Tibial",
                   "v1: Closing Meniscus","v1: Thick Femur X","v1: Thick Tibial X","v1: Thick Meniscus X",
                   "v2: Femur (vol)","v2: JWS (approx)", "v2: Img Intensity (mean)", "v2: Tibial Intensity (mean)", "v2: Femur Intensity (mean)", 
                   "v2: Miniscus Intensity (mean)", "v2: Entropy Tibial", "v2: Entropy Meniscus","v2: Entropy Femur","v2: Closing Femur","v2: Closing Tibial",
                   "v2: Closing Meniscus","v2: Thick Femur X","v2: Thick Tibial X","v2: Thick Meniscus X",
                   "v3: Femur (vol)","v3: JWS (approx)", "v3: Img Intensity (mean)", "v3: Tibial Intensity (mean)", "v3: Femur Intensity (mean)", 
                   "v3: Miniscus Intensity (mean)", "v3: Entropy Tibial", "v3: Entropy Meniscus","v3: Entropy Femur","v3: Closing Femur","v3: Closing Tibial",
                   "v3: Closing Meniscus","v3: Thick Femur X","v3: Thick Tibial X","v3: Thick Meniscus X", 'y']
#var_of_interest = ["v1: Patella (vol)","v1: Medial Tibial (vol)","v1: Lateral Tibial (vol)","v1: Medial Femur (vol)",
 #                 "v1: Lateral Femur (vol)","v1: Medial Miniscus (vol)","v1: Lateral Miniscus (vol)","v1: JWS (approx)", 
  #                "v1: Img Intensity (mean)","v1: Femur Intensity (median)","v1: Miniscus Intensity (median)","v1: Entropy Tibial",
   #               "v1: Entropy Meniscus","v1: Entropy Femur","v1: Closing Femur","v1: Closing Tibial","v1: Closing Meniscus",
    #              "v1: Thick Femur X","v1: Thick Femur Y","v1: Thick Femur Z","v1: Thick Tibial Y","v1: Thick Tibial Z",
     #             "v1: Thick Meniscus X","v1: Thick Meniscus Y",
      #            "v2: Patella (vol)","v2: Medial Tibial (vol)","v2: Lateral Tibial (vol)","v2: Medial Femur (vol)",
       #           "v2: Lateral Femur (vol)","v2: Medial Miniscus (vol)","v2: Lateral Miniscus (vol)","v2: JWS (approx)", 
        #          "v2: Img Intensity (mean)","v2: Femur Intensity (median)","v2: Miniscus Intensity (median)","v2: Entropy Tibial",
         #         "v2: Entropy Meniscus","v2: Entropy Femur","v2: Closing Femur","v2: Closing Tibial","v2: Closing Meniscus",
          #        "v2: Thick Femur X","v2: Thick Femur Y","v2: Thick Femur Z","v2: Thick Tibial Y","v2: Thick Tibial Z",
           #       "v2: Thick Meniscus X","v2: Thick Meniscus Y",
            #      "v3: Patella (vol)","v3: Medial Tibial (vol)","v3: Lateral Tibial (vol)","v3: Medial Femur (vol)",
             #     "v3: Lateral Femur (vol)","v3: Medial Miniscus (vol)","v3: Lateral Miniscus (vol)","v3: JWS (approx)",
              #    "v3: Img Intensity (mean)","v3: Femur Intensity (median)","v3: Miniscus Intensity (median)","v3: Entropy Tibial",
               #   "v3: Entropy Meniscus","v3: Entropy Femur","v3: Closing Femur","v3: Closing Tibial","v3: Closing Meniscus",
                #  "v3: Thick Femur X","v3: Thick Femur Y","v3: Thick Femur Z","v3: Thick Tibial Y","v3: Thick Tibial Z",
                 # "v3: Thick Meniscus X","v3: Thick Meniscus Y",'y']
    
df = df[var_of_interest]
df["id"]=df.index
#df=df.set_index("id")

### Separate into val and train set

In [ ]:
TRAIN = pd.read_csv("matched_15032022", sep=",", header=0)
TRAIN = TRAIN.set_index("Unnamed: 0")
TRAIN = TRAIN.index

VAL = pd.read_csv("validation_15032022", sep=",", header=0)
VAL = VAL.set_index("Unnamed: 0")
VAL = VAL.index

In [ ]:
print("All: ", df.shape )

df["id"] = [int(i[:7]) for i in df.index]

# Select validation or training mode VAL or TRAIN
train = df.loc[ df['id'].isin(TRAIN)]
val = df.loc[~df['id'].isin(TRAIN)]

train = train.drop(["id"],axis=1)
val = val.drop(["id"],axis=1)

print("Train", train.shape)
print("Val", val.shape)
train.shape[0]+val.shape[0]

y = train["y"]
train= train.drop(["y"],axis=1)

y_val = val["y"]
val= val.drop(["y"],axis=1)

In [ ]:
X= np.array(train)
X_val= np.array(val)

# correlation

In [ ]:
import seaborn as sns
corr = train.iloc[:, :-1].corr(method='spearman')

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(20, 15))
sns.heatmap(corr, mask=mask, cmap="RdBu", vmin=-1, vmax=1, center=0, linewidths=.5)

fig.suptitle('Correlation matrix of Image Features', fontsize=15)
ax.text(0.77, 0.2, 'aegis4048.github.io', fontsize=13, ha='center', va='center',
         transform=ax.transAxes, color='grey', alpha=0.7)

fig.tight_layout()

### Define Stratified k-Fold CV

In [ ]:
# make stratified KFOLD 
n_split=10

skf = StratifiedKFold(n_splits=n_split, shuffle=False)

# Random Forest

In [ ]:
# make stratified KFOLD 
n_split=10
mean = 0
data =[]
preds_stat= np.array([])

for train_index, test_index in skf.split(X, y):
    Xtrain = X[train_index]
    ytrain = y[train_index]
    Xtest = X[test_index]
    ytest = y[test_index]
    
    #normalize data 
    m = np.mean(Xtrain,axis=0)
    std = np.std(Xtrain, axis =0)
    Xtrain = np.array((Xtrain - m )/std)
    Xtest = np.array((Xtest - m )/std)
    
    reg = RandomForestClassifier(max_depth=20).fit(Xtrain, ytrain)
    ypred = reg.predict(Xtest)
#    ypred = np.where(ypred<0,0, ypred)

    preds_stat = np.concatenate([preds_stat,ypred])

    print("Accuracy Score: ", roc_auc_score(ytest.astype(float), ypred))
    #print("Accuracy Score!: ",accuracy_score(ytest, ypred.round()))
    mean += roc_auc_score(ytest.astype(float), ypred)
    data.append(roc_auc_score(ytest.astype(float), ypred))
    
    
print("Mean accuracy score: ", mean/n_split)
print("SSE: ", get_standard_diviation(data) )
print("All: ", data)

In [ ]:
np.savetxt('RF_mri.csv', preds_stat, delimiter=',')   
np.savetxt('y_mri.csv', y, delimiter=',')   

### Validate

In [ ]:
m = np.mean(X,axis=0)
std = np.std(X, axis =0)
X = (X - m )/std
X_val = (X_val - m )/std

reg = RandomForestClassifier(max_depth=20).fit(X, y)
ypred = reg.predict(X_val)

print("Accuracy Score: ", roc_auc_score(y_val.astype(float), ypred))

# Kfold of Linear regression

In [ ]:
mean = 0
data =[]
preds_stat=np.array([])

for train_index, test_index in skf.split(X, y):
    Xtrain = X[train_index]
    ytrain = y[train_index]
    Xtest = X[test_index]
    ytest = y[test_index]
    
    #normalize data 
    m = np.mean(Xtrain,axis=0)
    std = np.std(Xtrain, axis =0)
    Xtrain = np.array((Xtrain - m )/std)
    Xtest = np.array((Xtest - m )/std)
    
    reg = LinearRegression().fit(Xtrain, ytrain)
    ypred = reg.predict(Xtest)
    ypred = np.where(ypred<0,0, ypred)
  
    preds_stat = np.concatenate([preds_stat,ypred])

    print("Accuracy Score: ", roc_auc_score(ytest.astype(float), ypred))
    mean += roc_auc_score(ytest.astype(float), ypred)
    data.append(roc_auc_score(ytest.astype(float), ypred))
    
print("Mean accuracy score: ", mean/n_split)
print("Confidence interval: ", get_standard_diviation(data))

In [ ]:
np.savetxt('MLinR_mri.csv', preds_stat, delimiter=',')   

### Validate

In [ ]:
#normalize
m = np.mean(X,axis=0)
std = np.std(X, axis =0)
X = (X - m )/std
X_val = (X_val - m )/std

reg = LinearRegression()
reg = reg.fit(X, y)
ypred = reg.predict(X_val)

print("Accuracy Score: ", roc_auc_score(y_val.astype(float), ypred))

## Make one big Linear regression

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y, test_size=0.20, random_state=True, shuffle=True)

m = np.mean(Xtrain,axis=0)
std = np.std(Xtrain, axis =0)
Xtrain = np.array((Xtrain - m )/std)
Xtest = np.array((Xtest - m )/std)

reg = LinearRegression().fit(Xtrain, ytrain)
ypred = reg.predict(Xtest)
ypred = np.where(ypred<0,0, ypred)
    
print("Accuracy Score: ", roc_auc_score(ytest.astype(float), ypred))
#print("Coefficients: \n", reg.coef_)
print("incept", reg.intercept_)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix( ytest,ypred.round())

In [ ]:
# Get corefficients
params = reg.coef_

# Add row to constants
newX = pd.DataFrame({"Constant":np.ones(len(Xtest))}).join(pd.DataFrame(Xtest))

# #calculate std 
m = np.mean(newX, axis=0)
n = len(newX)
std = np.sqrt(np.sum((newX-m)**2, axis=0)/(n))

#Add incepts
params = np.append(reg.intercept_,params)

t_values = params/std

# set se of incept to 0
std[0] = 0
std=np.array(std)
t_values[0] = 0
t_values = np.array(t_values)

myDF3 = pd.DataFrame()
myDF3["Coefficients"]= params
myDF3["Standard Errors"] =std
myDF3["t values"] = t_values
names=["incept"]+list(df.drop(columns=["y","id"]))
myDF3=myDF3.set_index(pd.Index(names))
myDF3

In [ ]:
importance = myDF3["t values"][1:]
n=len(importance)//3 #round((len(importance))/2)
# Get variables, connect to weights (#Note remove id and labels)
variables =[i[4:] for i in myDF3.index[1:]]

# plot histograms
plt.figure(figsize=(30, 10))
plt.bar([x for x in range(len(importance[:n]))], importance[:n], alpha=0.5, color="b",label="V1",edgecolor=None)
plt.xticks([x for x in range(len(importance[:n]))], variables[:n], rotation =90)
plt.bar([x for x in range(len(importance[n:2*n]))], importance[n:2*n], alpha=0.5, color="g",label="V2",edgecolor=None)
plt.xticks([x for x in range(len(importance[n:2*n]))], variables[n:2*n], rotation =90)
plt.bar([x for x in range(len(importance[2*n:]))], importance[2*n:], alpha=0.5, color="r",label="V3",edgecolor=None)
plt.xticks([x for x in range(len(importance[2*n:]))], variables[2*n:], rotation =90)
plt.plot(range(n),np.zeros(n),c="black")

plt.grid(True, linestyle='-.')
#plt.ylim([0,1])
#plt.yscale('symlog')
plt.tick_params(labelcolor='r', labelsize=20, width=3)
plt.savefig("allfeat")
plt.legend(loc="best",fontsize="xx-large")
plt.show()

## Logistic regression

In [ ]:
mean_list = []
preds_stat = np.array([])

mean = 0
for train_index, test_index in skf.split(X, y):
    Xtrain = X[train_index]
    ytrain = y[train_index]
    Xtest = X[test_index]
    ytest = y[test_index]
    
    m = np.mean(Xtrain,axis=0)
    std = np.std(Xtrain, axis =0)
    Xtrain = (Xtrain - m )/std
    Xtest = (Xtest - m )/std
    
    Log_reg = LogisticRegression(max_iter=20000).fit(Xtrain, ytrain)
    ypred = Log_reg.predict(Xtest)
    y_pred = Log_reg.predict_proba(Xtest)     

    preds_stat = np.concatenate([preds_stat,ypred])

    print("Accuracy Score: ", roc_auc_score(ytest.astype(float), y_pred[:,1]))
    mean +=roc_auc_score(ytest.astype(float), y_pred[:,1])
    mean_list.append(roc_auc_score(ytest.astype(float), y_pred[:,1]))
    
print("Mean accuracy score: ", mean/n_split)  
print("Confidence interval: ", get_standard_diviation(mean_list))

np.savetxt('MLogR_mri.csv', preds_stat, delimiter=',')   

### Validate

In [ ]:
#normalize
m = np.mean(X,axis=0)
std = np.std(X, axis =0)
X = (X - m )/std
X_val = (X_val - m )/std


#Pred
reg = LogisticRegression(max_iter=1000000000000000)
reg = reg.fit(X, y)
ypred = reg.predict(X_val)

print("Accuracy Score: ", roc_auc_score(y_val.astype(float), ypred))

# LASSO

In [ ]:
# make stratified KFOLD 
mean = 0
mean_list =[]
for train_index, test_index in skf.split(X, y):
    Xtrain = X[train_index]
    ytrain = y[train_index]
    Xtest = X[test_index]
    ytest = y[test_index]
    
    m = np.mean(Xtrain,axis=0)
    std = np.std(Xtrain, axis =0)
    Xtrain = (Xtrain - m )/std
    Xtest = (Xtest - m )/std
    
    reg_las = linear_model.Lasso(0.03)
    reg_las.fit(Xtrain, ytrain) 
    ypred = reg_las.predict(Xtest)

    print("Accuracy Score: ", roc_auc_score(ytest.astype(float), ypred))
    mean +=roc_auc_score(ytest.astype(float), ypred)
    mean_list.append(roc_auc_score(ytest.astype(float), ypred))

print("Mean accuracy score: ", mean/n_split)    
print("Confidence Interval: ", get_standard_diviation(mean_list) )

In [ ]:
# Get corefficients
params = reg_las.coef_

# Add row to constants
newX = pd.DataFrame({"Constant":np.ones(len(Xtest))}).join(pd.DataFrame(Xtest))

# #calculate std 
m = np.mean(newX, axis=0)
n = len(newX)
std = np.sqrt(np.sum((newX-m)**2, axis=0)/(n))

#Add incepts
params = np.append(reg_las.intercept_,params)

t_values = params/std

# set se of incept to 0
std[0] = 0
std=np.array(std)
t_values[0] = 0
t_values = np.array(t_values)

myDF3 = pd.DataFrame()
myDF3["Coefficients"]= params
myDF3["Standard Errors"] =std
myDF3["t values"] = t_values
names=["incept"]+list(df.drop(columns=["y","id"]))
myDF3=myDF3.set_index(pd.Index(names))
myDF3

In [ ]:
importance = myDF3["t values"][1:]
n=len(importance)//3#round((len(importance))/2)
# Get variables, connect to weights (#Note remove id and labels)
variables =[i[4:] for i in myDF3.index[1:]]

# plot histograms
plt.figure(figsize=(30, 10))
plt.bar([x for x in range(len(importance[:n]))], importance[:n], alpha=0.5, color="b",label="V1",edgecolor=None)
plt.xticks([x for x in range(len(importance[:n]))], variables[:n], rotation =90)
plt.bar([x for x in range(len(importance[n:2*n]))], importance[n:2*n], alpha=0.5, color="g",label="V2",edgecolor=None)
plt.xticks([x for x in range(len(importance[n:2*n]))], variables[n:2*n], rotation =90)
plt.bar([x for x in range(len(importance[2*n:]))], importance[2*n:], alpha=0.5, color="r",label="V3",edgecolor=None)
plt.xticks([x for x in range(len(importance[2*n:]))], variables[2*n:], rotation =90)
#plt.yscale('symlog')
plt.plot(range(0,n+1), np.zeros(n+1), c="black")
plt.grid(True, linestyle='-.')
#plt.ylim([-0.005,0.006])
plt.tick_params(labelcolor='r', labelsize=20, width=3)
plt.legend(loc="best",fontsize="xx-large")
plt.show()

### Validate

In [ ]:
reg = linear_model.Lasso(0.03)
reg = reg.fit(X, y)
ypred = reg.predict(X_val)

print("Accuracy Score: ", roc_auc_score(y_val.astype(float), ypred))